# Face Mask Detection using MobileNet

In [ ]:
#Defining our data generators
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os

img_rows, img_cols = 128,128
batch_size = 16

train_data_dir = './dataset/train'
validation_data_dir = './dataset/validation'

# Let's use some data augmentation and define our generators
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      shear_range=0.3,
      zoom_range=0.3,
      width_shift_range=0.4,
      height_shift_range=0.4,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

train_images = train_generator.samples
validation_images = validation_generator.samples

# Loading the MobileNet

In [ ]:
"""
    We will use the pretrained MobileNet to which we will add our own network of FC layers.
    We then train the new model formed for the additional layers.

"""
from keras.applications import MobileNet

# Lets load the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Layers are set to be trainable as True by default but lets make them untrainable
for layer in MobileNet.layers:
    layer.trainable = False
    
# Let's check our layers 
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
def addTopModelMobileNet(bottom_model):
    """
    
        Creates the head of the model that will be 
        placed ontop of the bottom layers
    
    """

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(64,activation='relu')(top_model)
    top_model = Dense(2,activation='sigmoid')(top_model)
    
    return top_model


#Combining the model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

FC_Head = addTopModelMobileNet(MobileNet)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

# Training the mask detection model

In [ ]:
#Training our model
#One could do some tweakings!
from keras.optimizers import RMSprop, SGD,Adadelta
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

                     
checkpoint = ModelCheckpoint("face_mask_detector.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 2, verbose = 1, min_delta = 0.0001)

callbacks = [earlystop, checkpoint, reduce_lr]

nb_train_samples = train_images
nb_validation_samples = validation_images
batch_size = 16
epochs = 20

model.compile(loss = 'categorical_crossentropy',
                  optimizer = 'rmsprop',
                  metrics = ['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)


# Testing the mask detection model

In [36]:
def detect_mask():
    #Test on realtime video
    from os import listdir
    from os.path import isfile, join
    import os
    import cv2
    import numpy as np
    from keras.models import load_model
    return_label = ''
    return_frame = ''
    classifier = load_model('face_mask_detector.h5')
    img_rows, img_cols = 128,128
    # A function that puts the predicted class lables on the parametric image frames
    def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
                   font_scale=0.8, thickness=1):
        size = cv2.getTextSize(label, font, font_scale, thickness)[0]
        x, y = point
        cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
        cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)

    #Define our prediction dictionary
    face_classes = {0: 'with_mask', 1: 'without_mask'}
    img_size = 128

    detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    cap = cv2.VideoCapture(0)

    while True:

        ret, frame = cap.read()
        return_frame = frame
        preprocessed_faces = []           

        input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_h, img_w, _ = np.shape(frame)
        detected = detector.detectMultiScale(frame)
        faces = np.empty((len(detected), img_size, img_size, 3))

        if len(detected) > 0:

            for f in detected:

                # Obtain the coordinates of the detected face and draw a bounding box
                x1, y1, w, h = [v for v in f]
                cv2.rectangle(frame, (x1, y1), (x1+w,y1+h), (255, 0, 0), 2)
                face =  frame[y1:y1+h, x1:x1+w, :]
                face = cv2.resize(face, (img_rows,img_cols))
                face = face.astype("float32") / 255.0
                face = np.expand_dims(face, axis=0)
                preprocessed_faces.append(face)


            # Make predictions for the detected face 
            face_labels = []
            for i, d in enumerate(detected):
                preds = classifier.predict(preprocessed_faces[i])[0]
                face_labels.append(face_classes[np.argmax(preds,axis=0)])
    #             print(preds)
            
            
            
            # Putting labels on frames
            for i, d in enumerate(detected):
                label = "{}".format(face_labels[i])
                draw_label(frame, (x1,y1), label)
                return_label = face_labels[i]
                

        #Display the results
        
        cv2.imshow("Face Mask Recognition", frame)
        if cv2.waitKey(1) == 13: #13 is the Enter Key
            break

    cap.release()
    cv2.destroyAllWindows()
    return return_label,return_frame

In [3]:
# #You can use some images for testing also using this part of the code
# from os import listdir
# from os.path import isfile, join
# import os
# import cv2
# import numpy as np

# # A function that puts the predicted class lables on the parametric image frames
# def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
#                font_scale=0.8, thickness=1):
#     size = cv2.getTextSize(label, font, font_scale, thickness)[0]
#     x, y = point
#     cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
#     cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)
    
# #Define our prediction dictionary
# face_classes = {0: 'with_mask', 1: 'without_mask'}
# img_size = 128

# detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
# frame = cv2.imread('E:\Others\Shimla Manali\IMG_20210910_134647.jpg')
# frame = cv2.resize(frame,(1080,720))
# preprocessed_faces = []           

# input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
# img_h, img_w, _ = np.shape(frame)
# detected = detector.detectMultiScale(frame)
# faces = np.empty((len(detected), img_size, img_size, 3))

# if len(detected) > 0:

#     for f in detected:

#         # Obtain the coordinates of the detected face and draw a bounding box
#         x1, y1, w, h = [v for v in f]
#         cv2.rectangle(frame, (x1, y1), (x1+w,y1+h), (255, 0, 0), 2)
#         face =  frame[y1:y1+h, x1:x1+w, :]
#         face = cv2.resize(face, (img_rows,img_cols))
#         face = face.astype("float32") / 255.0
#         face = np.expand_dims(face, axis=0)
#         preprocessed_faces.append(face)


#     # Make predictions for the detected face 
#     face_labels = []
#     for i, d in enumerate(detected):
#         preds = classifier.predict(preprocessed_faces[i])[0]
#         face_labels.append(face_classes[np.argmax(preds,axis=0)])
#         print(preds)

#     # Putting labels on frames
#     for i, d in enumerate(detected):
#         label = "{}".format(face_labels[i])
#         print(label)
#         draw_label(frame, (x1,y1), label)

# # Display the results
# cv2.imshow("Face Recognition", frame)
# cv2.waitKey(0) == 13


# cap.release()
# cv2.destroyAllWindows()      

In [37]:
def train_face():
    from imutils import paths
    import face_recognition
    import pickle
    import cv2
    import os

    #get paths of each file in folder named Images
    #Images here contains my data(folders of various persons)
    imagePaths = list(paths.list_images('dataset_face'))
    knownEncodings = []
    knownNames = []
    # loop over the image paths
    for (i, imagePath) in enumerate(imagePaths):
        # extract the person name from the image path
        name = imagePath.split(os.path.sep)[-2]
        # load the input image and convert it from BGR (OpenCV ordering)
        # to dlib ordering (RGB)
        image = cv2.imread(imagePath)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #Use Face_recognition to locate faces
        boxes = face_recognition.face_locations(rgb,model='hog')
        # compute the facial embedding for the face
        encodings = face_recognition.face_encodings(rgb, boxes)
        # loop over the encodings
        for encoding in encodings:
            knownEncodings.append(encoding)
            knownNames.append(name)
    #save emcodings along with their names in dictionary data
    data = {"encodings": knownEncodings, "names": knownNames}
    #use pickle to save data into a file for later use
    f = open("face_enc", "wb")
    f.write(pickle.dumps(data))
    f.close()

In [38]:
def detect_face(frame):
    import face_recognition
    import imutils
    import pickle
    import time
    import cv2
    import os

    #find path of xml file containing haarcascade file
    cascPathface = os.path.dirname(
     cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
    # load the harcaascade in the cascade classifier
    faceCascade = cv2.CascadeClassifier(cascPathface)
    # load the known faces and embeddings saved in last file
    data = pickle.loads(open('face_enc', "rb").read())
    #Find path to the image you want to detect face and pass it here
    image = frame
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #convert image to Greyscale for haarcascade
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)

    # the facial embeddings for face in input
    encodings = face_recognition.face_encodings(rgb)
    names = []
    # loop over the facial embeddings incase
    # we have multiple embeddings for multiple fcaes
    for encoding in encodings:
        #Compare encodings with encodings in data["encodings"]
        #Matches contain array with boolean values and True for the embeddings it matches closely
        #and False for rest
        matches = face_recognition.compare_faces(data["encodings"],
        encoding)
        #set name =inknown if no encoding matches
        name = "Unknown"
        # check to see if we have found a match
        if True in matches:
            #Find positions at which we get True and store them
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            # loop over the matched indexes and maintain a count for
            # each recognized face face
            for i in matchedIdxs:
                #Check the names at respective indexes we stored in matchedIdxs
                name = data["names"][i]
                #increase count for the name we got
                counts[name] = counts.get(name, 0) + 1
                #set name which has highest count
                name = max(counts, key=counts.get)


            # update the list of names
            names.append(name)
            # loop over the recognized faces
            for ((x, y, w, h), name) in zip(faces, names):
                # rescale the face coordinates
                # draw the predicted face name on the image
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(image, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                 0.75, (0, 255, 0), 2)
        cv2.imshow("Frame", image)
        cv2.waitKey(0)
        return names

In [39]:
def send_mail(name):
    mails = {'Mayank':'goel.mayank.13@gmail.com','Saksham':'saksham474011@gmail.com'}
    import smtplib

    gmail_user = 'tempcheckbtp@gmail.com'
    gmail_password = 'Testing@123'

    sent_from = gmail_user
    to = mails[name]
    subject = 'WARNING | FOUND VIOLATING COVID19 REGULATIONS'
    body = 'Hey\n You were found violating the COVID19 regulations in the work premises. Please contact your Manager.'

    email_text = """\
    From: %s
    To: %s
    Subject: %s

    %s
    """ % (sent_from, ", ".join(to), subject, body)

    try:
        smtp_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        smtp_server.ehlo()
        smtp_server.login(gmail_user, gmail_password)
        smtp_server.sendmail(sent_from, to, email_text)
        smtp_server.close()
        print ("Email sent successfully!")
    except Exception as ex:
        print ("Something went wrong….",ex)

In [40]:
import cv2
label,frame = detect_mask()
# print(label)
cv2.imwrite("frame.jpg",frame)
# detect_face_image(frame)
# cv2.waitKey(0)
if label == 'without_mask':
    print("Checking face")
    names = detect_face(frame)
    print("Name ",names)
    send_mail(names[0])



Checking face
Name  None


TypeError: 'NoneType' object is not subscriptable